In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/deliveries']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# ALEX: remove path printing
# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
temp = pd.read_csv("./input/deliveries.scaled.csv")

In [3]:
temp.columns[[1]]

Index(['inning'], dtype='object')

In [4]:
t = temp.drop(temp.columns[[3, 9, 15, 17, 18, 19, 20]], axis = 1)

In [5]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136598 entries, 0 to 136597
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   match_id      136598 non-null  int64 
 1   inning        136598 non-null  int64 
 2   batting_team  136598 non-null  object
 3   over          136598 non-null  int64 
 4   ball          136598 non-null  int64 
 5   batsman       136598 non-null  object
 6   non_striker   136598 non-null  object
 7   bowler        136598 non-null  object
 8   wide_runs     136598 non-null  int64 
 9   bye_runs      136598 non-null  int64 
 10  legbye_runs   136598 non-null  int64 
 11  noball_runs   136598 non-null  int64 
 12  penalty_runs  136598 non-null  int64 
 13  extra_runs    136598 non-null  int64 
dtypes: int64(10), object(4)
memory usage: 14.6+ MB


In [6]:
#t.ix[:,8:14].head(2)
#collist = list(t)
#collist.remove('match_id', 'inning', 'batting_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler')

t['extra_sum'] = t[['wide_runs','bye_runs','legbye_runs','noball_runs','penalty_runs','extra_runs']].sum(axis=1)
 

In [7]:
evaluate_lazy(t)

,match_id,inning,batting_team,over,ball,batsman,non_striker,bowler,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,extra_runs,extra_sum
0,1,1,Kolkata Knight Riders,1,1,SC Ganguly,BB McCullum,P Kumar,0,0,1,0,0,1,2
1,1,1,Kolkata Knight Riders,1,2,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0
2,1,1,Kolkata Knight Riders,1,3,BB McCullum,SC Ganguly,P Kumar,1,0,0,0,0,1,2
3,1,1,Kolkata Knight Riders,1,4,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0
4,1,1,Kolkata Knight Riders,1,5,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136593,577,2,Royal Challengers Bangalore,20,2,Sachin Baby,CJ Jordan,B Kumar,0,0,0,0,0,0,0
136594,577,2,Royal Challengers Bangalore,20,3,Sachin Baby,CJ Jordan,B Kumar,0,0,0,0,0,0,0
136595,577,2,Royal Challengers Bangalore,20,4,Iqbal Abdulla,Sachin Baby,B Kumar,0,0,1,0,0,1,2
136596,577,2,Royal Challengers Bangalore,20,5,Sachin Baby,Iqbal Abdulla,B Kumar,0,0,0,0,0,0,0


In [8]:
df1 = t.groupby(['bowler', 'batting_team', 'over' ,'match_id', 'inning'])

In [9]:
df2 = t.groupby(['bowler', 'non_striker']) #, 'over' ,'match_id', 'inning'])

In [10]:
evaluate_lazy(df2[['noball_runs']].count())

noball_runs
bowler         non_striker                 
A Ashish Reddy AB de Villiers            11
               AC Thomas                  2
               AD Mathews                 9
               AL Menaria                 2
               AM Nayar                   3
...                                     ...
Z Khan         WA Mota                    2
               WP Saha                   25
               Y Venugopal Rao            7
               YK Pathan                 16
               Yuvraj Singh               1

[15175 rows x 1 columns]

In [11]:
evaluate_lazy(df1['bowler'].count())

bowler          batting_team         over  match_id  inning
A Ashish Reddy  Chennai Super Kings  13    293       1         7
                                           492       2         6
                                     15    492       2         6
                                     19    293       1         6
                                     20    355       2         5
                                                              ..
Z Khan          Sunrisers Hyderabad  6     502       1         7
                                     17    569       1         6
                                     18    418       1         6
                                     19    569       1         6
                                     20    502       1         6
Name: bowler, Length: 22154, dtype: int64